In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import glob
import os.path
import re
import pandas as pd

import seaborn as sns
from latexify import latexify, format_axes

/home/fl350/.pyenv/versions/2.7.11/envs/jupyter/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


# Theanets RNN search

LSTM and GRU yield most promising performance, but require regularization

In [3]:
theanets_data = '/home/fl350/data/theanets_cv/'

In [4]:
import pickle
losses_t = pickle.load(open(theanets_data + 'losses_t.pkl', 'rb'))
losses_v = pickle.load(open(theanets_data + 'losses_v.pkl', 'rb'))

IOError: [Errno 2] No such file or directory: '/home/fl350/data/theanets_cv/losses_t.pkl'

In [ ]:
latexify(fig_height=2.5, columns=2)

fig, (ax1,ax2) = plt.subplots(1,2)

for i, form in enumerate(losses_t.keys()):
    if form in ['scrn']:
        continue
    if form == 'clockwork':
        lab = 'Clockwork'
    else:
        lab = form.upper()
    ax1.plot(losses_t[form], label=lab, alpha=0.7)
    ax2.plot(losses_v[form], label=lab, alpha=0.7)

for ax in [ax1, ax2]:
    ax.set_xlabel('Epoch')
    ax.legend(loc='best')
    if ax == ax1:
        ax1.set_ylabel('Training loss')
    else:
        ax2.set_ylabel('Validation loss')

format_axes(ax1)
format_axes(ax2)
fig.suptitle('Training curves for various RNN architectures')
fig.tight_layout()
fig.savefig('theanets-architecture.pgf', bbox_inches='tight', dpi=300)
fig.show()

# `torch-rnn` grid search

In [ ]:
torch_logs = '/home/fl350/data/torch_logs'

confs = dict()
train = dict()
val = dict()
for log_path in glob.glob(os.path.join(torch_logs, '*.log')):
    conf = dict()
    for param in re.split(r'[,\s]*', os.path.basename(log_path)[:-4]):
        k, v = param.split('=')
        conf[k] = float(v)
        
    train_list = []
    val_list = []
    with open(log_path, 'r') as log:
        prev_epoch = None
        for line in log:
            if line[:5] == 'Epoch':
                values = re.split(r'[,\s]*', line)
                train_list.append({
                        'epoch': float(values[1]),
                        'loss': float(values[-2])
                    })
                prev_epoch = train_list[-1]['epoch']
            if line[:3] == 'val':
                val_list.append({
                        'epoch': prev_epoch,
                        'loss': float(line.split()[-1])
                    })
    train[log_path] = pd.DataFrame(train_list, columns=['epoch', 'loss'])
    val[log_path] = pd.DataFrame(val_list, columns=['epoch', 'loss'])
    confs[log_path] = conf
confs = pd.DataFrame(confs).T

## Enumerate all performance results

In [ ]:
config_perfs = (confs
                .loc[[log_path for log_path in confs.index if len(val[log_path]['loss']) != 0]]
                .assign(val_metric=lambda df: map(lambda x: val[x]['loss'].iloc[-1], df.index),
                        train_metric=lambda df: map(lambda x: train[x]['loss'].iloc[-10:].mean(), df.index))
                .sort_values(['val_metric', 'train_metric']))
with open('torch-rnn-config-perfs.tex', 'wb') as f:
    f.write(
        config_perfs[[
                'num_layers',
                'rnn_size',
                'seq_length',
                'wordvec',
                'train_metric',
                'val_metric']].to_latex(index=False,longtable=True))

# Dropout improves generalization

In [ ]:
confs_subset = confs[
    confs['wordvec'] == 32][
    confs['seq_length'] == 128][
    confs['rnn_size'] == 256][
    confs['num_layers'] == 3]

latexify(fig_height=2.5, columns=2)
fig, (ax1,ax2) = plt.subplots(1,2)

for log_path in confs_subset.index:
    lab = 'dropout={}'.format(*(confs.loc[log_path][['dropout']]))
    
    train_subset = train[log_path][train[log_path]['epoch'] <= 50]
    ax1.plot(train_subset['epoch'], train_subset['loss'], label=lab)

    val_subset = val[log_path][val[log_path]['epoch'] <= 50]
    ax2.plot(val_subset['epoch'], val_subset['loss'], label=lab)

for ax in [ax1, ax2]:
    ax.set_xlabel('Epoch')
    ax.legend(loc='best')
    if ax == ax1:
        ax1.set_ylabel('Training loss')
    else:
        ax2.set_ylabel('Validation loss')
        
format_axes(ax1)
format_axes(ax2) 
fig.suptitle('Training curves for various dropout settings')
fig.tight_layout()
fig.savefig('torch-rnn-dropout.pgf', bbox_inches='tight', dpi=300)
fig.show()

## Sensitivity to network parameters `num_layers` and `rnn_size`

 * Larger `rnn_size` leads to higher capacity and lower training loss
     * Presents as overfitting on validation, where the lowest capacity model `rnn_size` appears to be improving in generalization while others are flat/increasing
 * Training curves about the same wrt `num_layers`, validation curves have interesting story
     * Depth matters: small `64` and `128` hidden unit RNNs saw improvements up to `0.09`
     * Expressivity gained from depth furthers overfitting: `256` hidden unit RNN has some of the best validation performance at depth `1` but is the worst generalizing model for depths `2` and `3` even though training loss is low
 * `rnn_size=128` undisputably best generalizing, optimized at `num_layers=2`: will continue with these settings

In [ ]:
confs_subset = confs[
    confs['wordvec'] == 32][
    confs['seq_length'] == 128][
    confs['dropout'] == 0.3][confs['rnn_size'] <= 256.0]

latexify(fig_height=2.5, columns=2)
fig, (ax1,ax2) = plt.subplots(1,2)

for log_path in confs_subset.index:
    lab = 'rnn\_size={0}, num\_layers={1}'.format(*(confs.loc[log_path][['rnn_size', 'num_layers']]))
    
    train_subset = train[log_path][train[log_path]['epoch'] <= 50]
    ax1.plot(train_subset['epoch'], train_subset['loss'], label=lab)

    val_subset = val[log_path][val[log_path]['epoch'] <= 50]
    ax2.plot(val_subset['epoch'], val_subset['loss'], label=lab)
    
for ax in [ax1, ax2]:
    ax.set_xlabel('Epoch')
    ax.legend(loc='best')
    if ax == ax1:
        ax1.set_ylabel('Training loss')
    else:
        ax2.set_ylabel('Validation loss')

format_axes(ax1)
format_axes(ax2)
fig.suptitle('Training curves for various hidden layer architectures')
fig.tight_layout()
fig.savefig('torch-rnn-network-params.pgf', bbox_inches='tight', dpi=300)
fig.show()

In [ ]:
confs_subset = confs[
    confs['wordvec'] == 32][
    confs['seq_length'] == 128][
    confs['dropout'] == 0.3][confs['num_layers'] == 3]

latexify(fig_height=2.5, columns=2)
fig, (ax1,ax2) = plt.subplots(1,2)

for log_path in confs_subset.index:
    lab = 'rnn\_size={0}'.format(*(confs.loc[log_path][['rnn_size']]))

    train_subset = train[log_path][train[log_path]['epoch'] <= 50]
    ax1.plot(train_subset['epoch'], train_subset['loss'], label=lab)

    val_subset = val[log_path][val[log_path]['epoch'] <= 50]
    ax2.plot(val_subset['epoch'], val_subset['loss'], label=lab)

for ax in [ax1, ax2]:
    ax.set_xlabel('Epoch')
    ax.legend(loc='best')
    if ax == ax1:
        ax1.set_ylabel('Training loss')
    else:
        ax2.set_ylabel('Validation loss')

format_axes(ax1)
format_axes(ax2)
fig.suptitle('Increasing rnn\_size does not improve performance')
fig.tight_layout()
fig.savefig('torch-rnn-network-params-rnn-size.pgf', bbox_inches='tight', dpi=300)
fig.show()

In [ ]:
confs_subset = confs[
    confs['wordvec'] == 32][
    confs['seq_length'] == 128][
    confs['dropout'] == 0.3][
    confs['rnn_size'] == 256]

latexify(fig_height=2.5, columns=2)
fig, (ax1,ax2) = plt.subplots(1,2)

for log_path in confs_subset.index:
    lab = 'num\_layers={0}'.format(*(confs.loc[log_path][['num_layers']]))

    train_subset = train[log_path][train[log_path]['epoch'] <= 50]
    ax1.plot(train_subset['epoch'], train_subset['loss'], label=lab)

    val_subset = val[log_path][val[log_path]['epoch'] <= 50]
    ax2.plot(val_subset['epoch'], val_subset['loss'], label=lab)

for ax in [ax1, ax2]:
    ax.set_xlabel('Epoch')
    ax.legend(loc='best')
    if ax == ax1:
        ax1.set_ylabel('Training loss')
    else:
        ax2.set_ylabel('Validation loss')

format_axes(ax1)
format_axes(ax2)
fig.suptitle('Increasing num\_layers does not improve performance')
fig.tight_layout()
fig.savefig('torch-rnn-network-params-num-layers.pgf', bbox_inches='tight', dpi=300)
fig.show()

## Sensitivity to network input parameters `seq_length` and `wordvec`

* Training losses are about the same across all `wordvec`s
* Validation losses suggest that increasing `seq_length` important for good performance (**TODO**: investigate further)
* `wordvec=128` overfits for all cases, the other two depend on `seq_length` and vary an order of magnitude smaller than the performance gains from increasing `seq_length`

In [ ]:
confs_subset = confs[
    confs['num_layers'] == 3][
    confs['rnn_size'] == 256][
    confs['dropout'] == 0.3][
    confs['seq_length'] <= 256][
    confs['wordvec'] >= 32]

latexify(fig_height=2.5, columns=2)
fig, (ax1,ax2) = plt.subplots(1,2)

for log_path in confs_subset.index:
    lab = 'seq\_length={0}, wordvec={1}'.format(*(confs.loc[log_path][['seq_length', 'wordvec']]))

    train_subset = train[log_path][train[log_path]['epoch'] <= 50]
    ax1.plot(train_subset['epoch'], train_subset['loss'], label=lab)

    val_subset = val[log_path][val[log_path]['epoch'] <= 50]
    ax2.plot(val_subset['epoch'], val_subset['loss'], label=lab)


for ax in [ax1, ax2]:
    ax.set_xlabel('Epoch')
    ax.legend(loc='best')
    if ax == ax1:
        ax1.set_ylabel('Training loss')
    else:
        ax2.set_ylabel('Validation loss')
   
format_axes(ax1)
format_axes(ax2)
fig.suptitle('Training curves for various network input configurations')
fig.tight_layout()
fig.savefig('torch-rnn-input-params.pgf', bbox_inches='tight', dpi=300)
fig.show()

In [ ]:
confs_subset = confs[
    confs['num_layers'] == 3][
    confs['rnn_size'] == 256][
    confs['dropout'] == 0.3][
    confs['seq_length'] == 128][
    confs['wordvec'] <= 64]

latexify(fig_height=2.5, columns=2)
fig, (ax1,ax2) = plt.subplots(1,2)

for log_path in confs_subset.index:
    lab = 'seq\_length={0}, wordvec={1}'.format(*(confs.loc[log_path][['seq_length', 'wordvec']]))

    train_subset = train[log_path][train[log_path]['epoch'] <= 50]
    ax1.plot(train_subset['epoch'], train_subset['loss'], label=lab)
    
    val_subset = val[log_path][val[log_path]['epoch'] <= 50]
    ax2.plot(val_subset['epoch'], val_subset['loss'], label=lab)


for ax in [ax1, ax2]:
    ax.set_xlabel('Epoch')
    ax.legend(loc='best')
    if ax == ax1:
        ax1.set_ylabel('Training loss')
    else:
        ax2.set_ylabel('Validation loss')
        
format_axes(ax1)
format_axes(ax2) 
fig.suptitle('Decreasing wordvec does not improve performance')
fig.tight_layout()
fig.savefig('torch-rnn-input-params-wordvec.pgf', bbox_inches='tight', dpi=300)
fig.show()

# Long training trace for best model

In [ ]:
confs_subset = confs[
    confs['num_layers'] == 3][
    confs['rnn_size'] == 256][
    confs['dropout'] == 0.3][
    confs['seq_length'] == 128][
    confs['wordvec'] == 32]

latexify(fig_height=2.5, columns=2)
fig, (ax1,ax2) = plt.subplots(1,2)

for log_path in confs_subset.index:
    ax1.plot(train[log_path]['epoch'], train[log_path]['loss'])
    ax2.plot(val[log_path]['epoch'], val[log_path]['loss'])

for ax in [ax1, ax2]:
    ax.set_xlabel('Epoch')
    ax.legend().remove()
    if ax == ax1:
        ax1.set_ylabel('Training loss')
    else:
        ax2.set_ylabel('Validation loss')

format_axes(ax1)
format_axes(ax2)
fig.suptitle('Full training curve for best model')
fig.tight_layout()
fig.savefig('torch-rnn-best-model-trace.pgf', bbox_inches='tight', dpi=300)
fig.show()

# Timing results

In [ ]:
# GPU iter time
pd.Series(
    [0.5139319896698, 0.51304793357849, 0.51326394081116, 0.51345205307007, 0.51674485206604, 0.51290202140808, 0.5130250453949, 0.51296401023865, 0.52149391174316, 0.51301789283752, 0.51289296150208, 0.51626086235046, 0.51311111450195, 0.51304578781128, 0.51297211647034, 0.51289987564087, 0.51781296730042, 0.51277709007263, 0.51297402381897, 0.51298689842224, 0.51370096206665, 0.51244211196899, 0.51295900344849, 0.51374816894531, 0.5130729675293, 0.5133011341095, 0.51282286643982, 0.5134859085083, 0.51265978813171, 0.51299977302551, 0.51464009284973, 0.51310110092163, 0.51305794715881, 0.51279497146606, 0.51300501823425, 0.51785707473755, 0.51303005218506, 0.51328802108765, 0.51295900344849, 0.51360106468201, 0.51292514801025, 0.51304817199707, 0.512864112854, 0.51331996917725, 0.51318192481995, 0.51314401626587, 0.51287794113159, 0.51308107376099, 0.51309204101562, 0.51287293434143, 0.51289010047913, 0.51318407058716, 0.51275110244751, 0.51298594474792, 0.51694798469543, 0.51284909248352, 0.513258934021, 0.51281905174255, 0.5132269859314, 0.51307821273804, 0.51284408569336, 0.51677298545837, 0.51278686523438, 0.51306200027466, 0.51294898986816, 0.51290106773376, 0.51293206214905, 0.51287078857422, 0.51549696922302, 0.51325511932373, 0.51352787017822, 0.51287388801575, 0.51354503631592, 0.51282215118408, 0.51311612129211, 0.51364588737488, 0.5125720500946, 0.51288390159607, 0.51279997825623, 0.51293706893921, 0.5160129070282, 0.51293897628784, 0.51292705535889, 0.51308417320251, 0.51312303543091, 0.51302695274353, 0.51296401023865, 0.51373219490051, 0.51289510726929, 0.51339292526245, 0.51267194747925, 0.51315808296204, 0.51300001144409, 0.51312208175659, 0.51321697235107, 0.51301217079163, 0.5129919052124, 0.5127420425415, 0.51318788528442, 0.51674318313599, 0.51355290412903, 0.51297903060913, 0.51813006401062, 0.51305818557739, 0.51363086700439, 0.51319217681885, 0.5129759311676, 0.51782298088074, 0.51316499710083, 0.51346492767334, 0.51319193840027, 0.51422882080078, 0.5132999420166, 0.51305913925171, 0.51421308517456, 0.51306009292603, 0.51313900947571, 0.51277995109558, 0.51337003707886, 0.51292896270752, 0.51276803016663, 0.51368188858032, 0.51321721076965, 0.51329779624939, 0.51295804977417, 0.5133068561554, 0.51606702804565, 0.51316404342651, 0.51302599906921, 0.51308703422546, 0.51311802864075, 0.51279306411743, 0.51326203346252, 0.51295614242554, 0.51317596435547, 0.51383519172668, 0.51337718963623, 0.51437878608704, 0.51310110092163, 0.51259398460388, 0.51303386688232, 0.51303696632385, 0.5143129825592, 0.51317286491394, 0.51308393478394, 0.51773810386658, 0.51321816444397, 0.51397585868835, 0.51324510574341, 0.51326584815979, 0.51314997673035, 0.51290202140808, 0.51780486106873, 0.51298689842224, 0.51307988166809, 0.51307988166809, 0.51326608657837, 0.5130090713501, 0.51308608055115, 0.5146279335022, 0.51330995559692, 0.51305890083313, 0.51357007026672, 0.51332116127014, 0.51566505432129, 0.5132429599762, 0.51504397392273, 0.51278281211853, 0.51330709457397, 0.5127911567688, 0.51332592964172, 0.51733493804932, 0.51325106620789, 0.51303219795227, 0.51291108131409, 0.5135669708252, 0.51315903663635, 0.51306080818176, 0.51561498641968, 0.51334309577942, 0.51315903663635, 0.51315903663635, 0.51449584960938, 0.51307392120361, 0.51322913169861, 0.51493000984192, 0.51327800750732, 0.51318383216858, 0.51326704025269, 0.51346302032471, 0.51752495765686, 0.51312208175659, 0.51496505737305, 0.51346206665039, 0.51333999633789, 0.51303601264954, 0.5129702091217, 0.51447701454163, 0.51311612129211, 0.51370406150818, 0.51416516304016, 0.51354694366455, 0.51430702209473, 0.52048110961914, 0.51459193229675, 0.51329708099365, 0.51312589645386, 0.51347589492798, 0.51334619522095, 0.51338815689087, 0.51305198669434, 0.51353406906128, 0.51419019699097, 0.51353907585144, 0.51377105712891, 0.5134391784668, 0.51430606842041, 0.51358985900879, 0.5134871006012, 0.51400303840637, 0.51345109939575, 0.51346492767334, 0.51316213607788, 0.5140380859375, 0.51354908943176, 0.51370692253113, 0.5164840221405, 0.51360011100769, 0.51333689689636, 0.51343202590942, 0.51330399513245, 0.51786804199219, 0.51338791847229, 0.51363301277161, 0.5131311416626, 0.51447796821594, 0.51337504386902, 0.51332592964172, 0.51368308067322, 0.51333808898926, 0.5141019821167, 0.51358509063721, 0.51325988769531, 0.51373600959778, 0.51328778266907, 0.5132269859314, 0.51346111297607, 0.51362895965576, 0.51360583305359, 0.51364517211914]
).describe()

In [ ]:
# CPU iter time
pd.Series(
    [3.9024810791016, 4.0740389823914, 5.0292918682098, 4.2756350040436, 4.8293030261993, 4.9587950706482, 3.9459891319275, 4.6268730163574, 4.3019380569458, 3.9980530738831, 4.1086218357086, 4.0222518444061, 4.2987909317017, 4.5710759162903, 4.2018940448761, 4.7302730083466, 4.5731360912323, 4.1016459465027, 5.2108597755432, 4.4234969615936, 4.5114510059357, 4.0404009819031, 4.4523429870605, 4.3799788951874, 4.7563149929047, 4.2937610149384, 4.0312750339508, 4.2867619991302, 5.0678050518036, 4.6078569889069, 4.7324869632721, 4.9551830291748, 4.2756679058075, 4.8664579391479, 4.1406090259552, 4.2760519981384, 4.1681292057037, 4.485426902771, 3.8979930877686, 4.4580779075623, 4.1580410003662, 5.2740890979767, 4.7918701171875, 4.2730689048767, 4.6782078742981, 4.4485988616943, 4.694543838501, 4.4640970230103, 4.2400779724121, 4.2421000003815, 4.7151169776917, 4.7024209499359, 3.9650158882141, 4.765949010849, 4.4240851402283, 4.5955970287323, 4.2823491096497, 5.0051510334015, 4.9924190044403, 4.7118949890137, 4.061311006546, 4.565446138382, 4.6633551120758, 4.7514989376068, 3.7477910518646, 3.7743000984192, 4.0330321788788, 4.2941019535065, 3.9500269889832, 4.5313789844513, 4.2680370807648, 4.0663189888, 4.4136869907379, 4.6696488857269, 4.4429478645325, 4.4172320365906, 4.5989208221436, 3.9081718921661, 4.5283210277557, 4.7758550643921, 5.1744129657745, 4.8110530376434, 5.231507062912, 4.5296671390533, 4.7598779201508, 4.0652959346771, 4.7742261886597, 4.8879389762878, 4.2686159610748, 4.1151521205902, 4.5466978549957, 4.0377411842346, 4.2836470603943, 4.3706741333008, 4.333976984024, 4.0504539012909, 4.0295689105988, 4.592474937439, 4.2300381660461, 4.1969459056854, 4.5851781368256, 4.1019430160522, 4.5428130626678, 5.5018539428711, 4.338790178299, 4.0079219341278, 4.1492102146149, 3.9424719810486, 4.3623359203339, 4.3210959434509, 4.3756248950958, 4.0437572002411, 4.1698870658875, 4.1548550128937, 4.7753150463104, 4.2989900112152, 4.2722780704498, 3.9935290813446, 4.3346650600433, 4.3288700580597, 3.9035749435425, 3.997759103775, 5.0494570732117, 4.9401228427887, 4.3752088546753, 4.4860711097717, 4.5056591033936, 4.2956719398499, 4.6135709285736, 4.0092499256134, 4.3140790462494, 3.977147102356, 4.6897211074829, 4.1199769973755, 4.3140571117401, 3.9214098453522, 5.1139590740204, 4.1146171092987, 4.4830219745636, 4.6121690273285, 4.6257119178772, 4.1316778659821, 4.7616069316864, 4.0765891075134, 4.1298339366913, 4.5778670310974, 4.7619118690491, 4.0012049674988, 4.2565989494324, 4.3359808921814, 3.8221290111542, 4.3097729682922, 4.112154006958, 3.9695827960968, 4.5841867923737, 4.792690038681, 4.0328071117401, 4.105721950531, 4.3309760093689, 4.1379768848419, 4.9597721099854, 5.0013098716736, 4.3186640739441, 3.9041018486023, 4.8659360408783, 4.1503109931946, 4.4808900356293, 4.8387479782104, 4.2567880153656, 4.102322101593, 4.130774974823, 4.0514090061188, 3.9323949813843, 4.8302760124207, 4.0501599311829, 4.555083990097, 4.5997078418732, 4.0456509590149, 4.5211429595947, 4.2899498939514, 4.1443631649017, 4.2364411354065, 4.3901290893555, 4.1659641265869, 4.5129268169403, 4.4135558605194, 4.6050400733948, 3.9418389797211, 4.0260679721832, 4.2903130054474, 3.931736946106, 3.9400508403778, 4.3117101192474, 4.0558009147644, 4.4280941486359, 5.029648065567, 4.4728090763092, 4.8479540348053, 4.1399030685425, 3.99640417099, 4.2147920131683, 4.1249539852142, 4.1391160488129, 4.1880991458893, 4.1159648895264, 3.8603990077972, 4.2501211166382, 5.0167129039764, 4.7717127799988, 4.2546877861023, 4.3578498363495, 4.0748779773712, 4.2060451507568, 3.9009971618652, 4.0012128353119, 4.0344409942627, 4.6631820201874, 4.4301221370697, 4.4274749755859, 4.2128591537476, 4.0518219470978, 4.8518068790436, 4.0039792060852, 4.8435099124908, 4.5538449287415, 3.9365079402924, 4.0840961933136, 3.848867893219, 4.8901798725128, 4.1727731227875, 4.2124760150909, 4.0415358543396, 4.1977961063385, 4.2098169326782, 3.9641849994659, 3.9192178249359, 4.0530848503113, 4.0083029270172, 4.1318230628967, 4.1164779663086, 4.3874108791351, 4.2478361129761, 4.5732259750366, 4.6959400177002, 5.0250508785248, 4.2985620498657, 4.3278229236603, 4.0089190006256, 4.5945448875427, 4.2041761875153, 4.2793338298798, 4.4482660293579, 4.4809868335724, 4.1613719463348, 4.031909942627, 4.078978061676, 4.2033081054688, 4.0500190258026, 4.1546230316162, 3.9581251144409, 4.008948802948, 4.5624759197235, 4.3652091026306, 4.4050951004028, 3.9467899799347, 3.7954540252686, 4.2854759693146, 4.2327098846436, 4.0102531909943, 4.2455759048462, 4.9476799964905, 4.3910660743713, 4.641016960144, 4.0573220252991, 4.4177329540253, 4.2584609985352, 4.21875, 3.9772078990936, 4.2989912033081, 4.0486199855804, 4.287605047226, 4.081563949585, 4.1243410110474, 4.3244788646698, 4.502032995224, 4.2488238811493, 4.8460230827332, 4.0197288990021, 4.1547310352325, 4.172651052475, 4.1664021015167, 4.6169509887695, 4.845911026001, 5.1754410266876, 4.6522850990295, 3.99724817276, 4.4073150157928, 4.2846660614014, 4.7287580966949, 4.7068531513214, 4.3656411170959, 4.4506838321686, 4.2518579959869, 5.226891040802, 4.3842179775238, 4.6417348384857, 4.0175089836121, 3.9814550876617, 3.9536430835724, 5.0562150478363, 4.1449599266052, 4.2285261154175, 4.2493298053741, 4.0821859836578, 4.2001571655273, 4.1676478385925, 4.2579109668732, 4.4653658866882, 3.8837530612946, 4.184830904007, 3.9278600215912, 4.3519639968872, 4.3787291049957, 3.8535099029541, 4.1068251132965, 4.3524770736694, 4.3375589847565, 4.7523698806763, 4.4881610870361, 4.4589660167694, 4.4315550327301, 4.0931868553162, 4.1898839473724, 4.2744488716125, 4.4816529750824, 4.238429069519, 4.2304501533508, 4.4717009067535, 4.1849839687347, 3.9918479919434, 4.0585141181946, 4.4687860012054, 4.1769001483917, 4.3679609298706, 4.1876580715179, 4.3632590770721, 4.5455071926117, 4.5196740627289, 4.0334939956665, 4.0737960338593, 3.9312999248505, 4.1449618339539, 3.8413569927216, 4.110365152359, 4.205069065094, 3.9169139862061, 3.9735469818115, 3.8647818565369, 4.0933520793915, 3.9826638698578, 4.0230581760406, 4.0768809318542, 3.9233169555664, 4.1656970977783, 3.9077529907227, 3.947056055069, 3.7664079666138, 4.1029829978943, 3.9111831188202, 3.9446749687195, 3.992830991745, 4.4510049819946, 4.0784859657288, 4.2285480499268, 3.9251692295074, 4.0016949176788, 4.1867201328278, 4.0837590694427, 4.0182988643646, 4.2581350803375, 4.2089171409607, 4.0228028297424, 4.0426981449127, 3.938138961792, 3.9172348976135, 4.4916100502014, 4.0404818058014, 3.9061839580536, 3.9638438224792, 4.3318572044373, 4.0331430435181, 3.9791090488434, 3.9070200920105, 4.0862259864807, 4.0968129634857, 3.9656300544739, 4.0568430423737, 4.169294834137, 4.3414359092712, 4.5069282054901, 4.4297790527344, 4.1935560703278, 4.485671043396, 4.4360830783844, 4.1740539073944, 4.0348739624023, 4.2724239826202, 4.7856819629669, 4.2204611301422, 4.2598929405212, 3.8674440383911, 4.0563638210297, 4.0578820705414, 4.4327960014343, 4.1809320449829, 4.1682651042938, 4.1347658634186, 4.1799349784851, 4.5186638832092, 4.0106041431427, 4.858638048172, 4.5389969348907, 4.3191168308258, 4.3194761276245, 4.1652119159698, 4.1316101551056, 4.0008850097656, 4.2972619533539, 4.6682140827179, 4.1740000247955, 4.5053379535675, 4.0884990692139, 4.2460210323334, 3.9837369918823, 4.1347899436951, 3.875097990036, 4.2756180763245, 4.2047641277313, 4.1703259944916, 4.0974199771881, 3.8737092018127, 4.1736211776733, 4.2238299846649, 4.1266667842865, 4.5958549976349, 4.0822288990021, 4.4029200077057, 4.6186621189117, 4.177120923996, 5.0372498035431, 4.0758829116821, 4.1814410686493, 4.0231010913849, 3.9077808856964, 3.918879032135, 3.9395368099213, 3.9049029350281, 4.3280208110809, 3.9185171127319, 3.803062915802, 3.9476099014282, 3.9268589019775, 4.0774748325348, 4.0989909172058, 4.5802199840546, 4.2204849720001, 4.2652640342712, 4.1054918766022, 4.2077910900116, 4.0195980072021, 3.9116430282593, 3.8387091159821, 3.9800817966461, 4.3709559440613, 4.5673589706421, 4.3500180244446, 4.2699418067932, 4.1073579788208, 4.1800510883331, 3.9951040744781, 4.1183989048004, 4.4106159210205, 4.0240440368652, 4.1416730880737, 4.0723350048065, 3.9793360233307, 4.0561499595642, 4.176815032959, 4.4503219127655, 4.1029391288757, 4.3301990032196, 4.1753330230713, 4.5592670440674, 3.9117920398712, 4.0489361286163, 3.917995929718, 4.1168050765991, 3.8979480266571, 4.0656971931458, 4.2329518795013]
).describe()